In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Etape 2 :
# Installer les bibliothèques nécessaires
!pip install easyocr
!pip install tensorflow
!pip install datasets
!pip install nltk
!pip install pandas
!pip install opencv-python-headless  # Pour éviter les conflits avec cv2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

#fabrication du dataset partiel

In [ ]:
import os
import pandas as pd
from datasets import load_from_disk
from tqdm import tqdm  # Pour la barre de progression

#-------------------------------------------------------------------------------
# Paramétrage de lancement
#-------------------------------------------------------------------------------
# Chemin pour trouver les datasets et enregistrer le CSV
results_dir = '/content/drive/MyDrive/formation Datascientest/RVL-CDIP/'
# Choix du dataset : 'val', 'train' ou 'test'
type_dataset = 'train'

# Suffixe du dataset
dataset_suffixName = '_dataset_ID'

train_csv_path = "/content/drive/MyDrive/formation Datascientest/jul24_bds_extraction/ETAPE 3/notebooks/image_IDs_batch_8.csv"

# Chemin de sauvegarde du dataset réduit
reduced_train_dataset_path = "/content/drive/MyDrive/formation Datascientest/RVL-CDIP/train_dataset_08"
#-------------------------------------------------------------------------------

# Charger le dataset
train_dataset = load_from_disk(os.path.join(results_dir, type_dataset + dataset_suffixName))

# Charger les CSVs contenant les IDs échantillonnés
train_ids_df = pd.read_csv(train_csv_path)

# Convertir les IDs en un set pour une recherche rapide
train_ids_set = set(train_ids_df['image_ID'])

# Obtenir les indices des exemples à sélectionner avec une barre de progression
train_indices = [i for i, example in tqdm(enumerate(train_dataset), total=len(train_dataset), desc="Processing Dataset")
                 if example['image_ID'] in train_ids_set]

# Sélectionner les indices dans les datasets
reduced_train_dataset = train_dataset.select(train_indices)

# Afficher la taille du dataset réduit
print(f"Taille du reduced_train_dataset : {len(reduced_train_dataset)} exemples")

# Sauvegarder le dataset réduit
reduced_train_dataset.save_to_disk(reduced_train_dataset_path)

print("Reduced datasets saved successfully!")


# Traitement OCR des images train






In [ ]:
!cp "/content/drive/MyDrive/formation Datascientest/RVL-CDIP/ocr_train08_01_dataset_ID.csv" "/content/result"

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm
import cv2
import easyocr
import nltk
from PIL import Image, UnidentifiedImageError
from datasets import load_from_disk
from concurrent.futures import ThreadPoolExecutor, as_completed
import io
import pandas as pd
import time
import threading

#-------------------------------------------------------------------------------
#paramétrage de lancement
#-------------------------------------------------------------------------------
#chemin pour trouver les datasets en enregistrer le csv
results_dir = '/content/drive/MyDrive/formation Datascientest/RVL-CDIP/'

# Nombre de threads pour le multithreading
num_threads = 16
#répertoire des OCR par image
temp_dir = '/content/tempreducedtrain/'

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Chemin du fichier CSV pour sauvegarder les résultats OCR
csv_path = '/content/result/' +'ocr_train08_01_dataset_ID.csv'

dataset_path = os.path.join(results_dir, 'train_dataset_08')
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------

start_time = time.time()
lock = threading.Lock()

if os.path.exists(temp_dir):
    # Get the current time and format it
    current_time = time.strftime("%Y%m%d-%H%M%S")
    # Rename the existing directory with the current time
    new_temp_dir = temp_dir[:-1] + "_" + current_time + '/'
    os.rename(temp_dir, new_temp_dir)

# Create the temp directory
os.makedirs(temp_dir)

# Configurer TensorFlow pour utiliser le GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU configuré pour l'utilisation.")



# Charger le DataFrame existant si le fichier CSV existe
if os.path.exists(csv_path):
    df_ocr = pd.read_csv(csv_path)
    print(f"Chargement du fichier CSV existant : {csv_path}")
else:
    # Initialiser un DataFrame vide si le fichier n'existe pas
    df_ocr = pd.DataFrame(columns=['image_ID', 'texte_de_ocr', 'confiance_de_ocr', 'label_de_image'])
    print("Aucun fichier CSV trouvé, création d'un nouveau DataFrame.")

# Chargement des datasets
dataset = load_from_disk(dataset_path)

# Fonction pour filtrer les images du label
def filter_images(index):
    try:
        example = dataset[index]
        return example['image'], example['label'], example['image_ID']
    except Exception as e:
        print(f"[WARNING] Erreur lors de la récupération de l'image à l'index {index}: {e}")
    return None, None, None



# Filtrer les indices pour le traitement
indices = list(range(len(dataset)))

# Utiliser ThreadPoolExecutor pour le multithreading avec une barre de progression
print(f"début du filtrage des image")
label_invoice_images = []
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(filter_images, i) for i in indices]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Filtrage des images", unit="image"):
        result, label, image_ID= future.result()
        if result is not None:
            label_invoice_images.append((result, label, image_ID))

print(f"Nombre d'images filtrées : {len(label_invoice_images)}")

# Fonction de prétraitement de l'image avec OpenCV
def preprocess_image(image_bytes):
    try:
        # Charger l'image avec PIL pour gérer le format .tif
        image = Image.open(io.BytesIO(image_bytes))
        image = image.convert('RGB')  # Convertir en RGB pour compatibilité OpenCV

        # Convertir l'image en un tableau NumPy
        image_np = np.array(image)

        # Redimensionnement
        image_resized = cv2.resize(image_np, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)

        # Conversion en niveaux de gris
        gray = cv2.cvtColor(image_resized, cv2.COLOR_RGB2GRAY)

        # Seuillage adaptatif
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

        # Suppression du bruit (médian)
        blur = cv2.medianBlur(thresh, 3)

        # Encoder l'image traitée en bytes pour OCR
        _, buffer = cv2.imencode('.png', blur)
        preprocessed_image_bytes = buffer.tobytes()

        return preprocessed_image_bytes

    except Exception as e:
        print(f"Erreur de prétraitement sur l'image: {e}")
        return None

# Initialiser le lecteur EasyOCR
reader = easyocr.Reader(['en'], gpu=True)

# Fonction pour appliquer l'OCR avec EasyOCR
def apply_ocr(image_bytes):
    try:
        # Convertir les bytes en une image pour EasyOCR
        image_np = np.array(Image.open(io.BytesIO(image_bytes)))
        result = reader.readtext(image_np, detail=1)

        # Préparer les données à retourner
        texts = []
        confidences = []
        coordinates = []
        for (coord, text, confidence) in result:
            texts.append(text)
            confidences.append(confidence)
            coordinates.append(coord)

        return texts, confidences, coordinates

    except UnidentifiedImageError:
        print(f"[WARNING] Impossible d'identifier le fichier image. Ignorer...")
        return [], [], []
    except Exception as e:
        print(f"[ERROR] Erreur lors de l'application de l'OCR: {e}")
        return [], [], []

# Liste pour stocker les résultats
ocr_results = []

# Charger les noms d'image déjà traités pour éviter les doublons
images_deja_traitees = set(df_ocr['image_ID'])
print(f"Nombre d'images déjà traitées : {len(images_deja_traitees)}")

# Appliquer le prétraitement et l'OCR sur toutes les images filtrées après prétraitement avec multithreading
print(f"Début de l'OCR sur {len(label_invoice_images)} images")
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = {}
    for image, label, image_ID in label_invoice_images:
        # ID de l'image
        image_name = image_ID

        # Vérifier si l'image a déjà été traitée
        if image_name in images_deja_traitees:
            #print(f"Image {image_name} déjà traitée, passage...")
            continue

        # Convertir l'image en bytes directement si ce n'est pas déjà le cas
        if isinstance(image, Image.Image):
            image_bytes = io.BytesIO()
            image.save(image_bytes, format='TIFF')
            image_bytes = image_bytes.getvalue()
        else:
            image_bytes = image

        preprocessed_image_bytes = preprocess_image(image_bytes)
        if preprocessed_image_bytes:
            future = executor.submit(apply_ocr, preprocessed_image_bytes)
            futures[future] = (image_name, label, preprocessed_image_bytes)

    for future in tqdm(as_completed(futures), total=len(futures), desc="Prétraitement et OCR des images", unit="image"):
        image_name, label, preprocessed_image_bytes = futures[future]
        texts, confidences, coordinates = future.result()

        # Enregistrer les résultats dans la liste
        ocr_results.append({
            'image_ID': image_name,
            'texte_de_ocr': " ".join(texts),
            'confiance_de_ocr': np.mean(confidences),
            'label_de_image': label
        })

        # Sauvegarder les résultats dans le DataFrame existant et enregistrer le fichier CSV
        new_row = pd.DataFrame([{
            'image_ID': image_name,
            'texte_de_ocr': " ".join(texts),
            'confiance_de_ocr': np.mean(confidences),
            'label_de_image': label
        }])
        temp_file = os.path.join(temp_dir, f"temp_{image_name}.csv")
        new_row.to_csv(temp_file, header=False, index=False)

print("Tous les résultats de l'OCR ont été enregistrés dans "+ csv_path)
end_time = time.time()
processing_time = end_time - start_time
print(f"Temps de traitement total : {round(processing_time / 60, 2)} minutes")

GPU configuré pour l'utilisation.
Chargement du fichier CSV existant : /content/result/ocr_train08_01_dataset_ID.csv
début du filtrage des image


Filtrage des images:   0%|          | 0/32000 [00:00<?, ?image/s]

Nombre d'images filtrées : 32000
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Nombre d'images déjà traitées : 22205
Début de l'OCR sur 32000 images


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:917: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:1424.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


Prétraitement et OCR des images:   0%|          | 0/9795 [00:00<?, ?image/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Tous les résultats de l'OCR ont été enregistrés dans /content/result/ocr_train08_01_dataset_ID.csv
Temps de traitement total : 143.28 minutes


#réunion de tous les fichiers du train

In [ ]:
!cp "/content/result/ocr_train08_02_dataset_ID.csv" "/content/drive/MyDrive/formation Datascientest/RVL-CDIP"

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

#-------------------------------------------------------------------------------
#paramétrage de lancement
#-------------------------------------------------------------------------------
#chemin pour trouver les datasets en enregistrer le csv
results_dir = '/content/result/'
# Nombre de threads pour le multithreading
num_threads = 40
temp_dir = '/content/tempreducedtrain/'
csv_path = results_dir +'ocr_train08_02_dataset_ID.csv'

#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------

# Vérifier si le répertoire results_dir existe et le créer s'il n'existe pas
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# Liste de tous les fichiers CSV dans le répertoire
fichiers_csv = [f for f in os.listdir(temp_dir) if f.endswith('.csv')]

# Nom du fichier de sortie
fichier_sortie = csv_path

# Définir les en-têtes
colonnes = ['image_ID', 'texte_de_ocr', 'confiance_de_ocr', 'label_de_image']

# Charger le fichier de sortie existant s'il existe
df_concatene = pd.DataFrame(columns=colonnes)

# Traiter les fichiers par lots
taille_lot = 1000  # Nombre de fichiers à traiter par lot

for i in tqdm(range(0, len(fichiers_csv), taille_lot), desc="Concaténation des fichiers CSV"):
    dataframes = []
    for fichier in fichiers_csv[i:i + taille_lot]:
        chemin_fichier = os.path.join(temp_dir, fichier)
        df = pd.read_csv(chemin_fichier, header=None, names=colonnes, sep=',')
        dataframes.append(df)

    # Concaténer les DataFrames du lot actuel
    df_lot = pd.concat(dataframes, ignore_index=True)

    # Ajouter les données concaténées au DataFrame existant
    df_concatene = pd.concat([df_concatene, df_lot], ignore_index=True)

    # Ajouter les données concaténées au fichier de sortie
    df_concatene.to_csv(fichier_sortie, header=True, index=False)



Concaténation des fichiers CSV:   0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-3-27738656af5f>:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concatene = pd.concat([df_concatene, df_lot], ignore_index=True)
Concaténation des fichiers CSV: 100%|██████████| 10/10 [00:15<00:00,  1.51s/it]


#nettoyage du trainocr 3 4 5

In [ ]:
import pandas as pd
import numpy as np
import ast

# Charger le fichier CSV existant
df = pd.read_csv('/content/drive/MyDrive/formation Datascientest/RVL-CDIP/ocr_train/pretraite/Ocr_train_image_IDs_batch_5.csv')


# Fonction pour convertir une chaîne de caractères représentant une liste en une vraie liste
def convertir_chaine_en_liste(chaine):
    try:
        # Convertir la chaîne de caractères en une liste Python
        return ast.literal_eval(chaine)
    except (ValueError, SyntaxError):
        # Si la conversion échoue, retourner NaN
        return np.nan

# Appliquer la fonction pour convertir les chaînes en listes dans 'confiance_de_ocr' et 'texte_de_ocr'
df['confiance_de_ocr'] = df['confiance_de_ocr'].apply(convertir_chaine_en_liste)
df['texte_de_ocr'] = df['texte_de_ocr'].apply(convertir_chaine_en_liste)

# Fonction pour transformer la liste texte_de_ocr en une seule chaîne de texte sans filtrage
def transformer_texte(liste_texte):
    if isinstance(liste_texte, list):
        # Concaténer tous les éléments de la liste en une seule chaîne sans filtrer les caractères
        texte_concatene = " ".join(liste_texte)
        return texte_concatene
    return np.nan

# Appliquer la fonction de transformation sur la colonne texte_de_ocr
df['texte_de_ocr'] = df['texte_de_ocr'].apply(transformer_texte)

# Fonction pour calculer la moyenne des valeurs de confiance
def calculer_moyenne_confiance(confiances):
    try:
        # Calculer la moyenne si la liste est non vide et que c'est bien une liste
        return np.mean(confiances) if isinstance(confiances, list) and len(confiances) > 0 else np.nan
    except:
        # En cas d'erreur de calcul (par exemple si ce n'est pas une liste), retourner NaN
        return np.nan

# Appliquer la fonction pour calculer la moyenne des valeurs de confiance
df['confiance_de_ocr'] = df['confiance_de_ocr'].apply(calculer_moyenne_confiance)

# Ne conserver que les colonnes pertinentes (image_ID, texte_de_ocr, moyenne_confiance)
df_final = df[['image_ID', 'texte_de_ocr', 'confiance_de_ocr', 'label_de_image']]


# Sauvegarder dans un nouveau fichier CSV
df_final.to_csv('/content/drive/MyDrive/formation Datascientest/RVL-CDIP/ocr_train/Ocr_train5.csv', index=False)

print("Transformation terminée et fichier sauvegardé sous 'fichier_output.csv'")


Transformation terminée et fichier sauvegardé sous 'fichier_output.csv'


# Réunion des fichiers OCR image en un seul fichier

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

#-------------------------------------------------------------------------------
#paramétrage de lancement
#-------------------------------------------------------------------------------
#chemin pour trouver les datasets en enregistrer le csv
results_dir = '/content/drive/MyDrive/formation Datascientest/RVL-CDIP/'
# Nombre de threads pour le multithreading
num_threads = 40
temp_dir = '/content/drive/MyDrive/formation Datascientest/RVL-CDIP/ocr_train'
csv_path = results_dir +'ocr_train_dataset_ID_complet.csv'

#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------

# Vérifier si le répertoire results_dir existe et le créer s'il n'existe pas
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# Liste de tous les fichiers CSV dans le répertoire
fichiers_csv = [f for f in os.listdir(temp_dir) if f.endswith('.csv')]

# Nom du fichier de sortie
fichier_sortie = csv_path

# Définir les en-têtes
colonnes = ['image_ID', 'texte_de_ocr', 'confiance_de_ocr', 'label_de_image']

# Charger le fichier de sortie existant s'il existe
df_concatene = pd.DataFrame(columns=colonnes)

# Traiter les fichiers par lots
taille_lot = 1000  # Nombre de fichiers à traiter par lot

for i in tqdm(range(0, len(fichiers_csv), taille_lot), desc="Concaténation des fichiers CSV"):
    dataframes = []
    for fichier in fichiers_csv[i:i + taille_lot]:
        chemin_fichier = os.path.join(temp_dir, fichier)
        df = pd.read_csv(chemin_fichier, header=None, names=colonnes, sep=',')
        dataframes.append(df)

    # Concaténer les DataFrames du lot actuel
    df_lot = pd.concat(dataframes, ignore_index=True)

    # Ajouter les données concaténées au DataFrame existant
    df_concatene = pd.concat([df_concatene, df_lot], ignore_index=True)

    # Ajouter les données concaténées au fichier de sortie
    df_concatene.to_csv(fichier_sortie, header=True, index=False)



Concaténation des fichiers CSV: 100%|██████████| 1/1 [00:15<00:00, 15.56s/it]


#nettoyage fichier final, on enlève  les entete en trop

In [ ]:
import csv

def nettoyer_csv(fichier_entree, fichier_sortie):
    # Ouvrir le fichier d'entrée en mode lecture
    with open(fichier_entree, 'r', newline='', encoding='utf-8') as entree:
        lecteur_csv = csv.reader(entree)

        # Ouvrir le fichier de sortie en mode écriture
        with open(fichier_sortie, 'w', newline='', encoding='utf-8') as sortie:
            ecrivain_csv = csv.writer(sortie)

            # Lire et écrire la première ligne (l'en-tête)
            en_tete = next(lecteur_csv)
            ecrivain_csv.writerow(en_tete)

            # Parcourir le reste du fichier et écrire uniquement les lignes qui ne sont pas des en-têtes
            for ligne in lecteur_csv:
                # Si la ligne n'est pas identique à l'en-tête, l'écrire dans le fichier de sortie
                if ligne != en_tete:
                    ecrivain_csv.writerow(ligne)

# Spécifier les fichiers d'entrée et de sortie
fichier_entree = '/content/drive/MyDrive/formation Datascientest/RVL-CDIP/ocr_train_dataset_ID_complet.csv'  # Remplace par le nom de ton fichier CSV
fichier_sortie = '/content/drive/MyDrive/formation Datascientest/RVL-CDIP/ocr_train_dataset_ID_propre.csv'     # Le nom du fichier de sortie

# Appeler la fonction pour nettoyer le CSV
nettoyer_csv(fichier_entree, fichier_sortie)

print(f"Nettoyage terminé ! Le nouveau fichier est sauvegardé sous le nom {fichier_sortie}.")


Nettoyage terminé ! Le nouveau fichier est sauvegardé sous le nom /content/drive/MyDrive/formation Datascientest/RVL-CDIP/ocr_train_dataset_ID_propre.csv.
